In [0]:
#Teamates
#1) Ravi Vaishnav
#2) Shreya Agrawal
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
#Define Convolution Neural Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3) #input -? OUtput? RF 
        self.conv2 = nn.Conv2d(8, 16, 3) 
        self.bnorm1 =  nn.BatchNorm2d(16)
        self.dout1 = nn.Dropout2d(p=0.2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3) 
        self.bnorm2 =  nn.BatchNorm2d(32)
        self.dout2 = nn.Dropout2d(p=0.1)
        self.conv4 = nn.Conv2d(32, 8, 1) 
        
        self.conv5 = nn.Conv2d(8, 16, 3) 
        self.bnorm1 =  nn.BatchNorm2d(16)
        self.dout1 = nn.Dropout2d(p=0.2)
        self.conv6 = nn.Conv2d(16, 32, 3) 
        self.bnorm2 =  nn.BatchNorm2d(32)
        self.dout2 = nn.Dropout2d(p=0.1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv7 = nn.Conv2d(32,8,1) 
        self.conv8 = nn.Conv2d(8,10,3)  

    def forward(self, x):
        
        x = F.relu(self.conv4(F.relu(self.dout2(self.bnorm2(F.relu(self.conv3(self.pool1(F.relu(self.dout1(self.bnorm1(self.conv2(F.relu(self.conv1(x))))))))))))))
        x = F.relu(self.conv7(self.pool2(F.relu(self.dout2(self.bnorm2(self.conv6(F.relu(self.dout1(self.bnorm1(self.conv5(x)))))))))))
        
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
            Conv2d-2           [-1, 16, 24, 24]           1,168
       BatchNorm2d-3           [-1, 16, 24, 24]              32
         Dropout2d-4           [-1, 16, 24, 24]               0
         MaxPool2d-5           [-1, 16, 12, 12]               0
            Conv2d-6           [-1, 32, 10, 10]           4,640
       BatchNorm2d-7           [-1, 32, 10, 10]              64
         Dropout2d-8           [-1, 32, 10, 10]               0
            Conv2d-9            [-1, 8, 10, 10]             264
           Conv2d-10             [-1, 16, 8, 8]           1,168
      BatchNorm2d-11             [-1, 16, 8, 8]              32
        Dropout2d-12             [-1, 16, 8, 8]               0
           Conv2d-13             [-1, 32, 6, 6]           4,640
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2055, Accuracy: 9579/10000 (96%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2135, Accuracy: 9649/10000 (96%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.1759, Accuracy: 9720/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1726, Accuracy: 9728/10000 (97%)



loss=0.06970074027776718 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.64it/s]



Test set: Average loss: 0.1704, Accuracy: 9703/10000 (97%)

